In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor

1. Исследование данных для построения модели для предсказания.
Делать сразу бинарную классификацию бессысленно, так как точно предсказать хотя бы год рождения - маловероятно.
Поэтому будет построена регрессионная модель, предсказывающая год рождения абонента, 
а затем будет проверяться попало ли предзсказание в окрестность радиуса zone от реального значения.

In [4]:
# Функция оценки радиуса окрестности zone для нахождения значения, дающего наибольшее качество бинарной классификации
def scorer(y_predict, born_date, is_true):
    best_score = [0,0]
#     Перибираем значнеия до 20 лет. Погрешность больше уже нас не интересует
    for zone in np.linspace(0, 20, 51):
#         Находим абсолютную разницу между датами, вычисляем истинность предсказания
#         и проверяем совпадает ли оно с правильным значением классификатора
        score = ((abs(born_date - y_predict) <= zone)==is_true).mean()
#         Запонимаем и возвращаем наилучшее занчение
        if score>best_score[0]: best_score = [score, zone]
    return best_score

In [5]:
# Функция подготовки данных
def prepare_data(dataset_calls, dataset_numbers):
#     Создаём пустой DataFrame, именуем столбцы
    data = pd.DataFrame(columns=dataset_calls.columns)
    data = data.drop('call_date', axis=1)
#     Группируем по номерам и времени суток, тем самым избавляясь от дат. Данные суммируем
    data = dataset_calls.groupby(by=['number_id', 'part_date'], as_index=False).sum()
#     data = data.sort_values(by=['number_id', 'part_date'])
    data.number_id = data.number_id.astype('int64')
    data.part_date = data.part_date.astype('int64')
    data.count_call_in = data.count_call_in.astype('int64')
    data.count_call_out = data.count_call_out.astype('int64')
    data.count_sms_in = data.count_sms_in.astype('int64')
    data.count_sms_out = data.count_sms_out.astype('int64')
    data.all_duration = data.all_duration.astype('int64')
#     Создаём аналог dummies для разбития признаков по времени суток
    for col_type in ['count_call_in', 'count_call_out','count_sms_in','count_sms_out','all_duration', 'sum_price']:
        for i in range(4):
            data['{}_{}'.format(col_type,i)] = data[col_type][data.part_date==i]
#     Избавляемся от остаточных колонок        
    data.drop(columns=['part_date','count_call_in', 'count_call_out','count_sms_in','count_sms_out','all_duration', 'sum_price'], axis=1, inplace=True)
#     Заполняем полученные пропуски нулями и группирум по number_id, получаем данные для анализа
    data.fillna(0, inplace=True)
    data = data.groupby(by='number_id', as_index=False).sum()
#     Добавляем данные из таблицы numbers в текуший dataframe. От даты рождения оставляем только год
    data = data.merge(dataset_numbers, 'left', on='number_id')
    data.born_date = pd.to_datetime(data['born_date'])
    data.born_date = data.born_date.map(lambda x: x.year)
    return data

In [10]:
data_train_calls = pd.read_csv("train_calls.csv")
data_train_numbers = pd.read_csv("train_numbers.csv")
data_train_calls.head()

,number_id,call_date,part_date,count_call_in,count_call_out,count_sms_in,count_sms_out,all_duration,sum_price
0,237494,13.03.2013,3,3,7,0,6,814,0.96
1,90258,06.03.2013,2,1,0,0,0,91,0.00
2,430547,02.02.2013,1,1,1,0,0,35,0.05
3,346626,27.02.2013,2,4,5,2,0,408,0.19
4,430547,27.03.2013,2,1,0,0,0,88,0.00


In [11]:
data_train_numbers.head()

,number_id,born_date,is_true
0,53987,17.02.1951,1
1,56010,03.02.1982,0
2,67546,28.02.1970,0
3,86280,17.02.1987,0
4,90258,18.02.1960,0


In [12]:
# Готовим тренировочные данные
data_train = prepare_data(data_train_calls, data_train_numbers)
data_train.head()

,number_id,count_call_in_0,count_call_in_1,count_call_in_2,count_call_in_3,count_call_out_0,count_call_out_1,count_call_out_2,count_call_out_3,count_sms_in_0,...,all_duration_0,all_duration_1,all_duration_2,all_duration_3,sum_price_0,sum_price_1,sum_price_2,sum_price_3,born_date,is_true
0,53987,0.0,76.0,107.0,49.0,1.0,90.0,170.0,96.0,0.0,...,16.0,10161.0,10478.0,9829.0,0.05,7.07,15.25,17.92,1951,1
1,56010,0.0,198.0,313.0,48.0,0.0,167.0,250.0,62.0,0.0,...,0.0,35749.0,61323.0,18884.0,0.00,26.36,25.93,6.61,1982,0
2,67546,0.0,24.0,37.0,19.0,0.0,2.0,10.0,2.0,0.0,...,0.0,1414.0,3284.0,2005.0,0.00,0.22,0.62,0.16,1970,0
3,86280,0.0,241.0,223.0,95.0,0.0,288.0,322.0,101.0,0.0,...,0.0,137806.0,109703.0,74417.0,0.00,19.80,16.07,8.66,1987,0
4,90258,0.0,40.0,36.0,16.0,0.0,65.0,46.0,15.0,0.0,...,0.0,7619.0,8295.0,2101.0,0.00,4.18,3.11,1.01,1960,0


In [14]:
# Разделяем на тренироврочный и тестовый наборы
subdate_train, subdate_test = train_test_split(data_train, test_size=0.7, random_state=77)
subdate_train.sort_index(inplace=True)
subdate_test.sort_index(inplace=True)
subdate_train.reset_index(inplace=True, drop=True)
subdate_test.reset_index(inplace=True, drop=True)
# Для обучения нас интересуют только данные классифицированные как True
# Создаём список с number_id с известным годом рождения
true_num = data_train_numbers['number_id'][data_train_numbers['is_true']==1].reset_index(drop=True)
# Оставляем только те данные тренировочного набора, номера которых входят в true_num
data_train_true = subdate_train.loc[subdate_train['number_id'].isin(true_num)]
data_train_true.reset_index(inplace=True, drop=True)
# Отделяем столбец целевого признака от тренировочного набора и от тестового
y_train_true = data_train_true.born_date
y_train_true = y_train_true.astype('float')
data_train_true = data_train_true.drop(['number_id', 'is_true', 'born_date'], axis=1)
y_test = subdate_test[['number_id', 'is_true', 'born_date']]
subdate_test.drop(['number_id', 'is_true', 'born_date'], axis=1, inplace=True)

In [15]:
# Нормируем тренировочные и тестовые данные
scaller = MinMaxScaler()
data_train_true = scaller.fit_transform(data_train_true)
subdate_test = scaller.transform(subdate_test)
# Создаём самопальный GridSearchCV для работы с функцией scorer
loss = ['ls', 'lad', 'huber', 'quantile']
max_feature = ['auto', 'sqrt', 'log2', None]
n_estimatirs = range(50,500,50)
best = [[0,0],0,0,0]
for g_loss in loss:
    for g_max in max_feature:
        for g_est in n_estimatirs:
            model_gbr = GradientBoostingRegressor(loss=g_loss, max_features=g_max, n_estimators=g_est)
            model_gbr.fit(data_train_true, y_train_true)
            pr = scorer(model_gbr.predict(subdate_test), y_test.born_date, y_test.is_true)
            if pr[0]>best[0][0]: best = [pr, g_loss, g_max, g_est]
            print(pr, g_loss, g_max, g_est)
# Проверяем различные комбинации, выводим наилучшие параметры для модели регрессии и параметра zone            
print('best result is: ', best)

[0.5428571428571428, 12.8] ls auto 50
[0.5428571428571428, 14.0] ls auto 100
[0.5428571428571428, 12.8] ls auto 150
[0.5428571428571428, 14.4] ls auto 200
[0.55, 14.0] ls auto 250
[0.5357142857142857, 13.200000000000001] ls auto 300
[0.5428571428571428, 14.0] ls auto 350
[0.5428571428571428, 13.200000000000001] ls auto 400
[0.55, 14.4] ls auto 450
[0.5357142857142857, 16.0] ls sqrt 50
[0.5285714285714286, 13.200000000000001] ls sqrt 100
[0.5357142857142857, 14.0] ls sqrt 150
[0.5285714285714286, 12.0] ls sqrt 200
[0.5357142857142857, 14.4] ls sqrt 250
[0.5428571428571428, 14.4] ls sqrt 300
[0.5214285714285715, 17.2] ls sqrt 350
[0.5285714285714286, 0.4] ls sqrt 400
[0.5142857142857142, 16.0] ls sqrt 450
[0.5428571428571428, 18.400000000000002] ls log2 50
[0.5142857142857142, 17.2] ls log2 100
[0.5285714285714286, 14.4] ls log2 150
[0.5214285714285715, 0.4] ls log2 200
[0.5357142857142857, 14.8] ls log2 250
[0.5357142857142857, 16.8] ls log2 300
[0.5214285714285715, 14.4] ls log2 350
[0

2.1. Создаём модель с полученными параметрами

In [18]:
data_train_calls = pd.read_csv("train_calls.csv")
data_train_numbers = pd.read_csv("train_numbers.csv")

In [19]:
# Готовим тренировочные данные с полным набором
data_train = prepare_data(data_train_calls, data_train_numbers)

In [20]:
# Задаём параметры модели
model = GradientBoostingRegressor(loss='quantile',max_features='sqrt',n_estimators=250)
zone = 7.6

In [21]:
# Аналогичным образом формируем тренировочные данные и выделяем целевой признак
true_num = data_train_numbers['number_id'][data_train_numbers['is_true']==1].reset_index(drop=True)
data_train_true_all = data_train.loc[data_train['number_id'].isin(true_num)]
data_train_true_all.reset_index(inplace=True, drop=True)
y_train_true_all = data_train_true_all.born_date
y_train_true_all = y_train_true_all.astype('float')
data_train_true_all = data_train_true_all.drop(['number_id', 'is_true', 'born_date'], axis=1)
# Применяем нормирование и обучаем модель
scaller = MinMaxScaler()
# data_train_true_all = scaller.fit_transform(data_train_true_all)
model.fit(scaller.fit_transform(data_train_true_all), y_train_true_all)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='quantile', max_depth=3,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=250,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

2.2. Применяем модель для предсказания на тестовых данных

In [22]:
data_test_calls = pd.read_csv("test_calls.csv")
data_test_numbers = pd.read_csv("test_numbers.csv")

In [23]:
# Готовим тестовые данные
data_test = prepare_data(data_test_calls, data_test_numbers)
# Нормируем в отдельный nparray тестовые данные
data_test_norm = scaller.transform(data_test.drop(columns=['number_id','born_date'], axis=1))
# Создаём столбец с предсказанными значениями
data_test['predict'] = model.predict(data_test_norm)
# Сравниваем предсказание года регрессией с введённой датой рождения, формируем результирующий стоблец классификатора
data_test['is_true'] = (abs(data_test.born_date-data_test.predict)<=zone)
# Добавляем итоговый столбец в таблицу numbers и сохраняем в файле
data_test_numbers = data_test_numbers.merge(data_test[['number_id','is_true']], 'left', on='number_id')
data_test_numbers.to_csv("test_numbers_out.csv")